# Master equations

In [ ]:
# file: seeq/master.py
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import LinearOperator
import scipy.integrate

## Superoperators and pseudo-states

This module deals with the study of Markovian master equations
$$i\partial_t\hat\rho = L_t\hat\rho$$
defined by a Lindblad superoperator that acts onto density matrices
$$L_t\hat\rho = -i[H_t,\hat\rho] + \sum_i \gamma_i \left[A_i \hat\rho B_i - \frac{1}{2}(B_i A_i \hat\rho + \hat\rho B_i A_i)\right].$$

Our density matrices will be Hermitian operators defined on a finite-dimensional Hilbert space. Given an orthonormal basis of states $\{|i\rangle\}_{i=1}^d,$ we can write
$$\hat\rho = \sum_{ij} \rho_{ij} |i\rangle\langle j|.$$

Since master equations are linear, it is convenient to think about the density matrices as elements in a vector space. In Quantum Optics it is customary to denote this as some pseudo-quantum-states living in a $\mathbb{C}^{d\times d}$ complex space
$$|\rho) := \sum_{ij} \rho_{ij} |i,j\rangle =: \sum_{r=1}^{d^2} \bar{\rho}_r |r)$$
with
$$\bar{\rho}_{id+j} := \rho_{ij}.$$

With this mapping, the Lindblad operator can be represented as a matrix.
$$L\hat \rho \to \bar{L}\bar{\rho},$$
with the superoperator matrix
$$\bar{L} = -i (H\otimes \mathbb{1} - \mathbb{1} \otimes H^T) + \sum_i \gamma_i \left[A_i \otimes B_i^T - \frac{1}{2}(B_iA_i)\otimes\mathbb{1} - \frac{1}{2}\mathbb{1}\otimes(B_iA_i)^T\right].$$

## Lindblad class

We define a linear superoperator that acts on density matrices. It assumes a Lindblad operator structure, with a (possibly time-dependent) Hamiltonian and some constant dissipators.

In [ ]:
# file: seeq/master.py

class Lindblad(LinearOperator):
    
    def __init__(self, H, dissipators=[], time=0.0, dimension=None):
        if dimension is None:
            if callable(H):
                raise Exception('In Lindblad, you must provide a dimension of the Hilbert space')
            dimension = H.shape[0]
        super(Lindblad, self).__init__(np.complex128, (dimension**2,dimension**2))
        self.Hamiltonian = H
        self.dissipators = dissipators
        self.dimension = dimension
        self.ρshape = (dimension,dimension)
        self.time = 0.0

    def apply(self, t, ρ):
        ρ = np.asarray(ρ)
        flat = ρ.ndim == 1
        if flat:
            ρ = ρ.reshape(self.ρshape)
        H = self.Hamiltonian
        if callable(H):
            Lρ = -1j * (H(t, ρ - H(t, ρ.conj().T).conj()))
        else:
            Lρ = -1j * (H @ ρ - ρ @ H)
        for (γi, Ai, Bi) in self.dissipators:
            Lρ += γi * (Ai @ ρ @ Bi - 0.5 * ((ρ @ Bi) @ Ai + Bi @ (Ai @ ρ)))
        return Lρ.flatten() if flat else Lρ

    def _matvec(self, ρ):
        return self.apply(self.time, ρ)

## Stationary state calculation

We want to solve the eigenvalue equation
$$L \rho = \lambda \rho$$
for the stationary case $\lambda=0$. If we rewrite $\rho$ as a complex vector $v_\rho \in \mathbb{C}^{d^2}$ and $L$ as a sparse matrix in $\mathbb{C}^{d^2\times d^2}$
$$L v_\rho = \lambda v_\rho$$
this equation has the following properties:

* All eigenvalues have negative or zero real part, $\mathrm{Re}(\lambda)\leq 0$, because the Lindblad operator is contractive.

* Not all solutions of this equation are valid density matrices, that is, if we reshape the vector $v_\rho$ into a density matrix form, it might not even be Hermitian. However, the Hermitian part of it, will satisfy the equation.

The fact that the superoperator $L$ has a kernel complicates matter quite a bit. There are many strategies in the literature to solve for the stationary state $Lv_\rho = 0$.

* Use a reduce-invert method to compute the eigenvalue $\lambda$ that is closest to a positive number $\lambda_r > 0$. The idea is that $(L-\lambda_r)$ will be a stable operator without zero eigenvalues and the quadratic form $(L-\lambda_r)^\dagger (L-\lambda_r)$ has a minimum at the vector $v_\rho$ that we seek. This method is implemented using Scipy's `eigs` routines.

* If we know that the kernel of $L$ has dimension 1 (only one zero eigenvector), we can simultaneously solve the system of equations $$L\rho = 0,~\mathrm{and}~\mathrm{tr}(\rho)=1$$ One way to do that is to find two vectors: an $x$ such that $x^\dagger v_\rho = \mathrm{tr}\rho$, and a random vector $y$ such that we can introduce $A = L - y x^\dagger$ and solve the system of equations $$(L - y x^\dagger) v_\rho = y.$$

* Similar to the one above, but what we do is replace one row in $L$ with the vector $x$ such that $x^\dagger v_\rho = \mathrm{tr}\rho$ and solve $L' v_\rho = y$, where $y$ is a vector of zeros except for the row in which $x$ was inserted.

In [ ]:
# file: seeq/master.py

def stationary_state(L, guess=None, method='eigs', tol=10**(-8), maxiter=1000):
    #
    # Compute the stationary state of a master equation using a variety
    # of methods:
    #
    #  - L : a Lindblad operator class that implements a method
    #    L.Lindblad() that returns the Linbdlad superoperator in
    #    matrix representation.
    #
    #  - guess : a guess for the density matrix. It may be either
    #    a vector, representing a pure state, or a density matrix.
    #
    #  - method : which method use to solve the equation
    #      SOLVE_EIGS = compute the zero-decay eigenstate of the
    #                   Lindblad operator using Arpack
    #
    #  - observables: return a list of expected values over the
    #    computed density matrix
    #
    d = L.dimension
    
    if guess is not None:
        if guess.size == d:
            # This is a pure state, make a density matrix
            guess = np.reshape(guess, (d,1))
            guess = guess @ guess.T.conjugate()
            guess /= np.trace(guess)
        guess = np.reshape(guess, (d*d,))

    def replace(vρ):
        #
        # This internal function creates a linear system of
        # equations that consists of 'd*d-1' rows corresponding
        # to the lindblad operator plus one row corresponding
        # to the trace of the density matrix. We have to solve
        #        A * vρ = rhs
        # where 'rhs' is zeros except for the row corresponding
        # to the trace, which is '1'.
        ρ = vρ.reshape(d,d)
        Lvρ = (L @ ρ).flatten()
        Lvρ[-1] = np.trace(ρ)
        return Lvρ

    if method == 'eigs':
        #
        # Compute one (k=1) eigenstate of the Lindblad operator which is
        # closest to the eigenvalue sigma=1 Since all eigenvalues of the
        # Lindblad operator have zero or negative real part, the closest
        # one is exactly the zero eigenstate.
        #
        value, vρ = sp.linalg.eigs(L, k=1, maxiter=maxiter, tol=tol,
                                   sigma=1, v0=guess)
        vρ = vρ.flatten()
    elif method == 'replace':
        vρ, info = sp.linalg.bicgstab(LinearOperator(L.ρshape, matvec=replace), rhs)
        if info > 0:
            raise Exception('Problem did not converge')
    else:
        raise Exception(f'Unknown method "{method}" in master.stationary_state()')
    #
    # Normalize the density matrix. Since the system of equations is linear,
    # any multiple of the solution is also a valid solution.
    ρ = np.reshape(vρ, (d,d))
    ρ /= np.trace(ρ)
    #
    # Even if we normalized, there may be some residual error and the
    # matrix might not be completely Hermitian. We check both
    Lvρ = L @ vρ
    λ = np.vdot(vρ, Lvρ) / np.vdot(vρ, vρ)
    ε = np.sum(np.abs(ρ - ρ.T.conjugate()))
    return ρ, [λ, ε]

## Examples

### Simple cavity

We create a trivial problem of a cavity with losses to test our ideas. The basic Hamiltonian is

$$H = \delta a^\dagger a + \Omega(a^\dagger + a)$$

This Hamiltonian is embedded in a master equation with losses:

\begin{eqnarray*}
\partial_t\rho &=& -i [H,\rho] + \kappa \mathcal{D}[a]\rho
\end{eqnarray*}

Here the Lindblad operators are defined as
$$\mathcal{D}[A]\rho = A \rho A^\dagger - \frac{1}{2}\{A^\dagger A, \rho\}$$

In [ ]:
from seeq.operators import boson_anihilation, boson_creation, boson_number

class Cavity(Lindblad):
    def __init__(L, δ=0.0, Ω=1, κ=0.3, nMax=5):
        #
        # Simple cavity problem
        #
        L.δ = δ        # cavity detuning
        L.Ω = Ω        # cavity driving Rabi frequency
        L.nMax = nMax  # max. number of photons in cavity
        L.κ = κ        # cavity decay
        
        # Fock space of the auxiliary "qubit" (second d.o.f of transmon)
        #   a = anihilation operator
        #   n = number operator
        L.a = boson_anihilation(nMax)
        L.adag = boson_creation(nMax)
        L.n = boson_number(nMax)
        super(Cavity, L).__init__(δ * L.n + Ω * (L.a + L.adag),
                                  [(L.κ, L.a, L.adag)])

    def vacuum(L):
        #
        # Return the vacuum state of the cavity with both qubits
        # in their ground state. Outputs a state vector, from which
        # a density matrix can be constructed
        #
        vac = np.zeros((L.dimension,))
        vac[0]=1.0
        return vac

The master equation for observables
$$\partial_t\langle{O}\rangle = -i \langle{[O,H]}\rangle + \frac{\kappa}{2}\langle{[a^\dagger, O]a + a^\dagger[O,a]}\rangle$$

can be particularized to two cases: (i) $O=a$
$$\partial_t\langle{a}\rangle = -i\delta \langle{a}\rangle -i \Omega -\frac{\kappa}{2}\langle{a}\rangle$$
and (ii) the number operator
$$\partial_t\langle{n}\rangle = -i\Omega\langle{a^\dagger-a}\rangle -\kappa\langle{a^\dagger a}\rangle$$

The stationary condition for the first equation
$$0 = \left(\delta-i\frac{\kappa}{2}\right)\langle{a}\rangle + \Omega$$

has as solution
$$\langle{a}\rangle = \frac{\Omega}{i\kappa/2 - \delta}$$
$$\langle{a^\dagger}\rangle = \frac{\Omega}{-i\kappa/2 - \delta}$$

Combined this gives
$$\langle{a^\dagger - a}\rangle = \frac{i\Omega\kappa}{\kappa^2/4+\delta^2}$$

and therefore
$$\langle{a^\dagger a}\rangle = -i\frac{\Omega}{\kappa}\langle{a^\dagger - a}\rangle = \frac{\Omega^2}{\kappa^2/4 + \delta^2}$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def test_Cavity(δ=0.0, κ=0.3):
    Ω = np.linspace(1e-3, 0.5)
    a = Ω * 0.0
    n = Ω * 0.0
    λ = Ω * 0.0
    ε = Ω * 0.0
    for i in range(Ω.size):
        L = Cavity(δ=δ, Ω=Ω[i], κ=κ, nMax = 10)
        ρ, err = stationary_state(L, guess = L.vacuum(), method = 'eigs')
        a[i] = abs(np.trace(L.a @ ρ))
        n[i] = np.trace(L.n @ ρ).real
        λ[i] = np.abs(err[0])
        ε[i] = err[1]
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,4))
    ax1.plot(Ω, a, 'r', label='$\\langle{a}\\rangle$')
    ax1.plot(Ω, np.abs(Ω/(0.5j*κ-δ)), 'r--', label='$\\langle{a}\\rangle$ theory')
    ax1.plot(Ω, n, 'b', label='$\\langle{a^\dagger a}\\rangle$')
    ax1.plot(Ω, np.abs(Ω**2/((0.5*κ)**2+δ**2)), 'b-.', label='$\\langle{a^\dagger a}\\rangle$ theory')
    ax1.set_ylim([0,4])
    ax1.legend()
    ax1.set_xlabel('$\\Omega$')
    ax2.plot(Ω, λ, label='$L$ eigenvalue')
    ax2.plot(Ω, ε, label='non-Hermiticity')
    ax2.legend()
    ax2.set_xlabel('$\\Omega$')

test_Cavity()